In [37]:
from absl import app
from nasbench import api

In [38]:
INPUT = 'input'
OUTPUT = 'output'
CONV1X1 = 'conv1x1-bn-relu'
CONV3X3 = 'conv3x3-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'

In [39]:
# Load the data from file (this will take some time)
nasbench = api.NASBench('./dataset/nasbench_full.tfrecord')

# Create an Inception-like module (5x5 convolution replaced with two 3x3
# convolutions).
model_spec = api.ModelSpec(
    # Adjacency matrix of the module
    matrix=[[0, 1, 1, 1, 0, 1, 0],    # input layer
            [0, 0, 0, 0, 0, 0, 1],    # 1x1 conv
            [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv
            [0, 0, 0, 0, 1, 0, 0],    # 5x5 conv (replaced by two 3x3's)
            [0, 0, 0, 0, 0, 0, 1],    # 5x5 conv (replaced by two 3x3's)
            [0, 0, 0, 0, 0, 0, 1],    # 3x3 max-pool
            [0, 0, 0, 0, 0, 0, 0]],   # output layer
    # Operations at the vertices of the module, matches order of matrix
    ops=[INPUT, CONV1X1, CONV3X3, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT])

# Query this model from dataset, returns a dictionary containing the metrics
# associated with this model.
data = nasbench.query(model_spec)

dict_keys(['absl.logging', 'absl.app', 'nasbench.lib.config', 'tensorflow.python.ops.parallel_for.pfor', 'tensorflow.python.tpu.client.client', 'absl.testing.absltest'])
Loading dataset from file... This may take a few minutes...


100%|██████████| 5083488/5083488 [01:56<00:00, 43650.51it/s]

Loaded dataset in 116 seconds


In [40]:
data

{'module_adjacency': array([[0, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0]], dtype=int8),
 'module_operations': ['input',
  'conv3x3-bn-relu',
  'conv1x1-bn-relu',
  'maxpool3x3',
  'conv3x3-bn-relu',
  'conv3x3-bn-relu',
  'output'],
 'trainable_parameters': 2694282,
 'training_time': 1157.675048828125,
 'train_accuracy': 1.0,
 'validation_accuracy': 0.9378004670143127,
 'test_accuracy': 0.932692289352417}

In [42]:
nasbench

In [45]:
%%python - --nouse_tpu --train_data_files ./nasbench/scripts/cifar-10-data/train_1.tfrecords,./nasbench/scripts/cifar-10-data/train_2.tfrecords,./nasbench/scripts/cifar-10-data/train_3.tfrecords,./nasbench/scripts/cifar-10-data/train_4.tfrecords --test_data_file ./nasbench/scripts/cifar-10-data/test.tfrecords --valid_data_file ./nasbench/scripts/cifar-10-data/validation.tfrecords

import sys
print(sys.argv)

nasbench.evaluate(model_spec, './test_output')

['-', '--nouse_tpu', '--train_data_files', './nasbench/scripts/cifar-10-data/train_1.tfrecords,./nasbench/scripts/cifar-10-data/train_2.tfrecords,./nasbench/scripts/cifar-10-data/train_3.tfrecords,./nasbench/scripts/cifar-10-data/train_4.tfrecords', '--test_data_file', './nasbench/scripts/cifar-10-data/test.tfrecords', '--valid_data_file', './nasbench/scripts/cifar-10-data/validation.tfrecords']


Traceback (most recent call last):
  File "<stdin>", line 5, in <module>
NameError: name 'nasbench' is not defined


CalledProcessError: Command 'b"\nimport sys\nprint(sys.argv)\n\nnasbench.evaluate(model_spec, './test_output')\n"' returned non-zero exit status 1.

In [60]:
# Preferable to run this on the command line
!python test.py --nouse_tpu --train_data_files ./nasbench/scripts/cifar-10-data/train_1.tfrecords,./nasbench/scripts/cifar-10-data/train_2.tfrecords,./nasbench/scripts/cifar-10-data/train_3.tfrecords,./nasbench/scripts/cifar-10-data/train_4.tfrecords --test_data_file ./nasbench/scripts/cifar-10-data/test.tfrecords --valid_data_file ./nasbench/scripts/cifar-10-data/validation.tfrecords

/usr/bin/zsh: /home/rockerbox/anaconda3/envs/nas_with_tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
2022-07-12 15:14:01.166806: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 15:14:01.169909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 15:14:01.169990: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
dict_keys(['absl.logging', 'absl.app', 'nasbench.lib.config', 'tensorflow.python.ops.parallel_for.pfor', 'tensorflow.python.tpu.client.client', 'absl.testing.absltest'])
Loading dataset from file... This 

In [33]:
from pathlib import Path

data_fps = Path('./') / 'nasbench' / 'scripts' / 'cifar-10-data'
data_fps = [str(fp) for fp in data_fps.iterdir() if fp.suffix == '.tfrecords']

list(map(lambda s: './' + s, data_fps))

['./nasbench/scripts/cifar-10-data/train_3.tfrecords',
 './nasbench/scripts/cifar-10-data/train_1.tfrecords',
 './nasbench/scripts/cifar-10-data/validation.tfrecords',
 './nasbench/scripts/cifar-10-data/test.tfrecords',
 './nasbench/scripts/cifar-10-data/train_2.tfrecords',
 './nasbench/scripts/cifar-10-data/sample.tfrecords',
 './nasbench/scripts/cifar-10-data/train_4.tfrecords']

In [56]:
%%python - --option1 ./value1,./value2 --option2 value2 --etc

import sys

if __name__ == '__main__':
    print(sys.argv)
    print(nasbench)

['-', '--option1', './value1,./value2', '--option2', 'value2', '--etc']


Traceback (most recent call last):
  File "<stdin>", line 6, in <module>
NameError: name 'nasbench' is not defined


CalledProcessError: Command 'b"\nimport sys\n\nif __name__ == '__main__':\n    print(sys.argv)\n    print(nasbench)\n"' returned non-zero exit status 1.

In [4]:
%%python - --option1 value1 --option2 value2 --etc

import test_cla_on_jupyter


Wa-hoo!
['-', '--option1', 'value1', '--option2', 'value2', '--etc']
